# Calibration - measured data

In [ ]:
from radiocalibrationtoolkit import *
from astropy.time import Time
from astropy import units as u
import matplotlib as mpl
import pickle
from pathlib import Path
from scipy.stats import bootstrap
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator
import matplotlib.dates as mdates
import datetime
from scipy.optimize import minimize

In [ ]:
# some global plot settings
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = 16
plt.rcParams["legend.fontsize"] = 14

plt.rcParams["xtick.major.width"] = 2
plt.rcParams["ytick.major.width"] = 2

plt.rcParams["xtick.major.size"] = 5
plt.rcParams["ytick.major.size"] = 5

plt.rcParams["xtick.labelsize"] = 14
plt.rcParams["ytick.labelsize"] = 14

layout_settings = dict(
    #title="<b>Measured power dataset: </b>",
    xaxis=dict(title="<b>LST</b>", tickprefix="<b>", ticksuffix="</b>", dtick=2),
    yaxis=dict(
        title="<b>frequency [MHz]</b>",
        tickprefix="<b>",
        ticksuffix="</b>",
        range=(30, 80),
        tick0=0,
        dtick=10,
        autorange=False,
    ),
    font=dict(
        size=20,
        color="black",
    ),
    coloraxis=dict(
        colorbar=dict(
            tickprefix="<b>",
            ticksuffix="</b>",
            title=dict(text="<b>Power [pW]</b>", side="right")),
        cmin=0,
        cmax=24,
    )
)

In [ ]:
# This ensures Plotly output works in multiple places:
# plotly_mimetype: VS Code notebook UI
# notebook: "Jupyter: Export to HTML" command in VS Code
# See https://plotly.com/python/renderers/#multiple-renderers
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"

In [ ]:
# some extra functions

def date_filtering(df, timePeriodCut=[0, np.inf]):
    if 'gpsTime' in df:
        return df[(df.gpsTime > timePeriodCut[0]) & (df.gpsTime < timePeriodCut[1])].reset_index(drop=True)
    else:
        print("[ERROR] No column called 'gpsTime'")

def encode_dataframes_to_latex(mean_df, neg_error_df, pos_error_df, overal_cal_params_df):
    # Compute the mean + poserr/-negerr values and format them with two decimal places
    formatted_values_df = (
        mean_df.applymap("${:.2f}".format)
        + neg_error_df.abs().applymap('_{{-{:.1f}\%}}'.format)
        + pos_error_df.abs().applymap('^{{+{:.1f}\%}}$'.format)
    )
    formatted_values_df = formatted_values_df.sort_index(axis=1)
    
    df1 = overal_cal_params_df.loc[:,'mu'].to_frame().T.applymap("${:.2f}".format)
    df1 = df1.reset_index(drop=True)
    overal_cal_params_rel_df = (overal_cal_params_df.div(overal_cal_params_df.loc[:,'mu'].values, axis=0) - 1).abs()*100
    df2 = overal_cal_params_rel_df.loc[:,'err_low'].to_frame().T.applymap('_{{-{:.1f}\%}}'.format)
    df2 = df2.reset_index(drop=True)
    df3 = overal_cal_params_rel_df.loc[:,'err_up'].to_frame().T.applymap('^{{+{:.1f}\%}}$'.format)
    df3 = df3.reset_index(drop=True)
    formatted_values_overall_df = df1+df2+df3
    formatted_values_overall_df = formatted_values_overall_df.sort_index(axis=1)
    formatted_values_overall_df.index = ['overall']
    
    final_df = pd.concat((formatted_values_df, formatted_values_overall_df))
    
    # Generate the LaTeX table
    latex_table = final_df.to_latex(
        header=True, escape=False, column_format="l"
    )

    return latex_table

def check_substring(string, substrings):
    for substring in substrings:
        if substring in string:
            return True
    return False

def convert_spectra2powers_and_concatenate_dfs(dir_path, timecut=[0, np.inf], channel='channel_0', exclude_list=[None]):

    df_files = [
        os.path.join(dir_path, i) for i in os.listdir(dir_path) if ((channel in i) & ~check_substring(i, exclude_list))
    ]
    df_list = []
    df_names = []
    for f in df_files:
        print(f)
        spectra_df = pd.read_csv(f, index_col=0)
        spectra_df = date_filtering(spectra_df, timePeriodCut=timecut)
        ###########################
        info_cols_df = spectra_df.iloc[:,:6]
        spectra_df = spectra_df.iloc[:,6:]
        spectra_df.columns = spectra_df.columns.astype(float)

        integrand_df = ((spectra_df)* (1 / (sampling_frequency_MHz*1e+6))**2).divide(
            impedance_func(spectra_df.columns.values)
        )

        # integrate
        rec_power_unbinned_DF = (2 / trace_time_length_sec) *integrate_spectral_density(
            integrand_df,
            # integrated_MHz_bands=np.linspace(0, 125, 126),
            integrated_MHz_bands=np.linspace(30, 80, 51),
            integrating_method='on_discontinuous_function',
        )

        rec_power_unbinned_DF = pd.concat((info_cols_df.loc[:,'lst'], rec_power_unbinned_DF), axis=1)

        power_rec_DF = bin_df_rows(rec_power_unbinned_DF, binning_column='lst', bins=list(range(25)))
        
        power_rec_DF.index.name = 'lst'
        power_rec_DF = power_rec_DF.drop(['lst'], axis=1) * 1e12
                
        ###########################
        df_list.append(power_rec_DF)
        df_names.append(Path(f).stem)

    return pd.concat(df_list, keys=df_names)

def filename2label(input_string):
    numbers = re.findall(r'\d+', input_string)
    ls_number = numbers[0] if numbers else ''
    ch_number = numbers[1] if len(numbers) >= 2 else ''

    new_string = f"LS:{ls_number} Ch:{ch_number}"
    return new_string


In [ ]:
######## SET ORIENTATION ########
orientation = 'EW'
if orientation == 'EW':
    channel = 'channel_0'
    exclude_list = ['noexclusion']
elif orientation == 'NS':
    channel = 'channel_1'
    exclude_list=["1733"]

## Data conversion to power dataset

In [ ]:
# system parameters
sampling_frequency_MHz = 250
ADC2Volts = 1/2048
N = 1024
trace_time_length_sec = N/(sampling_frequency_MHz*1e+6)

In [ ]:
# read HW response
hw_file_path = "./antenna_setup_files/HardwareProfileList_realistic.xml"
# hw_file_path = "./antenna_setup_files/HardwareProfileList_flat.xml"

hw_dict = read_hw_file(hw_file_path, interp_args={"fill_value": "extrapolate"})

# impedance function
impedance_func = hw_dict["IImpedance"]["antenna_"+orientation]

In [ ]:
# in voltage already
spectra_df = pd.read_csv(
    "/home/tomas/Documents/myRUthesis/thesis_codes/converted_f16/voltage_squared_spectra_31742_"+channel+".csv", index_col=0
)

In [ ]:
# startDay = Time('2021-12-29 00:00:00.000', format='iso')
startDay = Time('2022-01-03 00:00:00.000', format='iso')
endDay = startDay + 1*u.day
spectra_df = date_filtering(spectra_df, [startDay.gps, endDay.gps])

In [ ]:
info_cols_df = spectra_df.iloc[:,:6]
spectra_df = spectra_df.iloc[:,6:]
spectra_df.columns = spectra_df.columns.astype(float)

In [ ]:
integrand_df = ((spectra_df)* (1 / (sampling_frequency_MHz*1e+6))**2).divide(
    impedance_func(spectra_df.columns.values)
)

# integrate
rec_power_unbinned_DF = (2 / trace_time_length_sec) *integrate_spectral_density(
    integrand_df,
    # integrated_MHz_bands=np.linspace(0, 125, 126),
    integrated_MHz_bands=np.linspace(30, 80, 51),
    integrating_method='on_discontinuous_function',
)
rec_power_unbinned_DF = pd.concat((info_cols_df.loc[:,'lst'], rec_power_unbinned_DF), axis=1)

In [ ]:
power_rec_DF = bin_df_rows(rec_power_unbinned_DF, binning_column='lst', bins=list(range(25)))
power_rec_DF.index.name = 'lst'
power_rec_DF = power_rec_DF.drop(['lst'], axis=1) * 1e12

In [ ]:
power_rec_example_DF = power_rec_DF.copy(deep=True)

In [ ]:
fig = px.imshow(power_rec_DF.T, width=600, aspect="cube", color_continuous_scale="jet")
fig.update_layout(**layout_settings)
fig.update_layout(
    title="<b>Measured power dataset: </b>",
)
fig.show()

## Create list of simulated datasets

In [ ]:
antenna_type = 'Salla_'+orientation

dir_path = "./simulated_power_datasets/"
concatenated_sim_df = concatenate_simulated_dfs(dir_path, antenna_type)

# check keys
[key for key in concatenated_sim_df.index.levels[0]]

In [ ]:
sky_model = "GSM08"

fig = px.imshow(
    concatenated_sim_df.xs("Salla_" + orientation + "_" + sky_model).T,
    width=600,
    aspect="cube",
    color_continuous_scale="jet",
)
fig.update_layout(**layout_settings)
fig.update_layout(
    title="<b>Simulated power dataset: " + sky_model + " " + orientation + "</b>"
)
fig.show()

## Single calibration

In [ ]:
slopes_DF, intercepts_DF = get_fitted_voltage_cal_params_and_noise_offsets_from_concat_sim_dfs(concatenated_sim_df, power_rec_DF)
get_and_plot_calibration_results(slopes_DF, intercepts_DF, title="")

In [ ]:
power_sim_DF = concatenated_sim_df.loc['Salla_'+orientation + "_" + sky_model]

fig = px.imshow(
    power_sim_DF.add(
        intercepts_DF.loc["Salla_" + orientation + "_" + sky_model].values, axis=1
    ).T,
    width=600,
    aspect="cube",
    color_continuous_scale="jet",
)
fig.update_layout(**layout_settings)

fig.update_layout(
    title="<b>Simulated dataset + fitted noise</b>",
)
fig.show()


fig = px.imshow(
    power_rec_DF.sub(
        intercepts_DF.loc["Salla_" + orientation + "_" + sky_model].values, axis=1
    ).T,
    width=600,
    aspect="cube",
    color_continuous_scale="jet",
)
fig.update_layout(**layout_settings)

fig.update_layout(
    title="<b>Measured dataset - fitted noise</b>",
)
fig.show()

## Create list of measured datasets

In [ ]:
dir_path = "/home/tomas/Documents/myRUthesis/thesis_codes/converted_f16/"
#
if orientation == 'EW':
    channel = 'channel_0'
    exclude_list = ['noexclusion']
elif orientation == 'NS':
    channel = 'channel_1'
    exclude_list=["1733"]

concatenated_rec_df = convert_spectra2powers_and_concatenate_dfs(dir_path, timecut=[0, np.inf], channel=channel, exclude_list=exclude_list)

In [ ]:
concatenated_rec_df.index.levels[0]

## Calibration

In [ ]:
overal_cal_params_dict = {}
# labels = []
for key in concatenated_rec_df.index.levels[0]:
    print('**************************')
    print(filename2label(key))
    power_rec_DF = concatenated_rec_df.xs(key)
    slopes_DF, intercepts_DF = get_fitted_voltage_cal_params_and_noise_offsets_from_concat_sim_dfs(concatenated_sim_df, power_rec_DF)
    stats = get_and_plot_calibration_results(slopes_DF, intercepts_DF, title=filename2label(key))
    # fig, ax = plt.subplots()
    # _, _, stats = create_KDE_plot(
    #     truncate_data(dropnans(slopes_DF.values.flatten()), 5, 95), bins=np.linspace(0., 2, 3000)
    # )
    # labels.append(filename2label(key))
    overal_cal_params_dict[filename2label(key)] = stats
    print('**************************')

    # all_stats.append(stats)

overal_cal_params_df = pd.DataFrame(overal_cal_params_dict).T
overal_cal_params_df.columns = ['mu', 'err_low', 'err_up']

## Time dependent

In [ ]:
# super concating: time and stations
# we have 12 full days (until 9th of January)
startDay = Time("2021-12-29 00:00:00.000", format="iso")
dir_path = "/home/tomas/Documents/myRUthesis/thesis_codes/converted_f16/"

concat_dfs_per_day_dict = {}
for i in range(12):
    print(startDay)
    endDay = startDay + 1 * u.day
    concatenated_rec_df = convert_spectra2powers_and_concatenate_dfs(
        dir_path, timecut=[startDay.gps, endDay.gps], channel=channel, exclude_list=exclude_list
    )
    concat_dfs_per_day_dict[startDay.iso[:10]] = concatenated_rec_df
    startDay += 1 * u.day

In [ ]:
concatenated_rec_df.index.levels[0]

In [ ]:
# calibrating

concat_dfs_per_day_dict['2021-12-30']

nested_dict = lambda: collections.defaultdict(nested_dict)
means_dict = nested_dict()
lowererr_dict = nested_dict()
uppererr_dict = nested_dict()

for date in concat_dfs_per_day_dict.keys():
    print(date)

    all_stats = []
    labels = []
    concatenated_rec_df = concat_dfs_per_day_dict[date]
    for key in concatenated_rec_df.index.levels[0]:
        print('*******************************')
        print(filename2label(key))
        power_rec_DF = concatenated_rec_df.xs(key)
        slopes_DF, intercepts_DF = get_fitted_voltage_cal_params_and_noise_offsets_from_concat_sim_dfs(concatenated_sim_df, power_rec_DF)
        stats = get_frequency_independent_calibration_param(slopes_DF)
        # fig, ax = plt.subplots()
        # _, _, stats = create_KDE_plot(
        #     truncate_data(dropnans(slopes_DF.values.flatten()), 5, 95), bins=np.linspace(0., 2, 3000)
        # )
        # stats = plot_results_with_CI(slopes_DF, intercepts_DF, title=filename2label(key))
        labels.append(filename2label(key))
        all_stats.append(stats)
        means_dict[date][filename2label(key)] = stats[2]
        lowererr_dict[date][filename2label(key)] = stats[0]
        uppererr_dict[date][filename2label(key)] = stats[1]
        print('*******************************')

In [ ]:
means_df = pd.DataFrame(means_dict)
lowererr_df = pd.DataFrame(lowererr_dict)
uppererr_df = pd.DataFrame(uppererr_dict)

In [ ]:
color_cycle = plt.rcParams["axes.prop_cycle"].by_key()["color"]
fig, ax = plt.subplots()
xdate = [
    datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
    for date_str in means_df.columns.values
]
for i, row in enumerate(means_df.index):
    color = color_cycle[i % len(color_cycle)]
    y = means_df.loc[row, :].values
    yerr_low = lowererr_df.loc[row, :].values
    yerr_up = uppererr_df.loc[row, :].values

    plt.gca().set_prop_cycle(plt.rcParams["axes.prop_cycle"])
    ax.errorbar(
        xdate,
        y,
        yerr=[abs(yerr_low - y), abs(yerr_up - y)],
        label=row,
        color=color,
        marker="o",
        lw=0.5,
    )
    ax.axes.axhline(overal_cal_params_df.loc[row, "mu"], color=color)

ax.tick_params(axis="x", rotation=45)
ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m-%d"))

mu_total, err_low_total, err_up_total = overal_cal_params_df.mean()
ax.axes.axhline(
    mu_total,
    color="k",
    lw=3,
    label=r"$C_{{\mathrm{{overall}}}}={:.2f}_{{-{:.1f}\%}}^{{+{:.1f}\%}}$".format(
        mu_total,
        abs(err_low_total / mu_total - 1)*100,
        abs(err_up_total / mu_total - 1)*100,
    ),
)
ax.axes.axhspan(err_low_total, err_up_total, color="k", alpha=0.1)

ax.set_ylim(0.8, 1.25)
ax.set_ylabel("voltage calibration parameter")
ax.legend(loc="lower left", bbox_to_anchor=(1, 0.42))

In [ ]:
overal_cal_params_df.columns = ['mu', 'err_low', 'err_up']

In [ ]:
ltable = encode_dataframes_to_latex(
    means_df.T,
    np.abs(lowererr_df.T / means_df.T.values - 1) * 100,
    np.abs(uppererr_df.T / means_df.T.values - 1) * 100,
    overal_cal_params_df,
)

In [ ]:
display(means_df.T.sort_index(axis=1))
display(overal_cal_params_df.T)

print(ltable)
display('Universal channel calibration parameter:')
display(overal_cal_params_df.mean()[['mu']])
display('Its error in %:')
display((overal_cal_params_df.mean()/overal_cal_params_df.mean().mu - 1)[['err_low', 'err_up']])

## Test data type precisions

In [ ]:
spectra_f16_df = pd.read_csv(
    "/home/tomas/Documents/myRUthesis/thesis_codes/converted_f16/voltage_squared_spectra_30056_channel_0.csv", index_col=0
)
spectra_full_df = pd.read_csv(
    "/home/tomas/Documents/myRUthesis/thesis_codes/converted_full_precision/voltage_squared_spectra_30056_channel_0.csv", index_col=0
)



In [ ]:
fig, ax = plt.subplots()

ax.set_xlabel("frequency [MHz]")
ax.set_ylabel(
    r"$\frac{{\mathrm{{power \: from \: 64 \: int}}}}{{{}\: \mathrm{{power from \:bit \:float   }}}} - 1$".format(
        16
    )
)
ax.plot(
    spectra_full_df.columns[6:].values.astype(float),
    (spectra_full_df / spectra_f16_df.values).iloc[50, 6:].values - 1,
)

In [ ]:
print(*[Time(i, format='gps').iso+'\n' for i in spectra_f16_df.gpsTime])

# Test

In [ ]:
slopes_DF, intercepts_DF = get_fitted_voltage_cal_params_and_noise_offsets_from_concat_sim_dfs(concatenated_sim_df, power_rec_example_DF)
get_and_plot_calibration_results(slopes_DF, intercepts_DF, title="")

# Misc

## Signal decomposition

In [ ]:
# first create this dataset of results
overallNoiseFitResultDF = pd.concat((slopes_DF.mean(axis=0), intercepts_DF.mean(axis=0)), axis=1)
overallNoiseFitResultDF.columns=['S', 'Nf']
# and average simulated datasets
power_sim_avr_df = concatenated_sim_df.groupby(level=1).mean()

In [ ]:
from scipy import sparse
from scipy.sparse.linalg import spsolve

def _Baseline_als(y, lam=None, p=None, niter=10):
    L = len(y)
    D = sparse.diags([1, -2, 1], [0, -1, -2], shape=(L, L - 2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w * y)
        w = p * (y > z) + (1 - p) * (y < z)
    return z

def extract_baseline(dataset):
    newDataset = dataset.copy(deep=True)
    for i in range(dataset.index.size):
        arr = dataset.iloc[i, :].values
        newDataset.iloc[i, :] = _Baseline_als(
            arr, lam=500, p=0.1, niter=100
        )
    #                 self._newDataset.iloc[i,:] = baseline_als2(self._dataset.iloc[i,:].values, lam=100, lam1=0.01, p=0.1, niter=100)
    return newDataset

power_rec_baseline_DF = extract_baseline(power_rec_DF/ overallNoiseFitResultDF.S.values)

In [ ]:
def create_shared_subfigures(
    rows, cols, subfig_width, subfig_height, hspace=0.2, wspace=0.2
):
    fig, axes = plt.subplots(
        rows,
        cols,
        sharex=True,
        sharey=True,
        figsize=(subfig_width * cols, subfig_height * rows),
    )

    if rows == 1 & cols != 1:
        axes = axes.reshape(1, cols)
    elif rows == 1 & cols == 1:
        axes = np.array(axes).reshape(1, 1)

    plt.subplots_adjust(wspace=wspace, hspace=hspace)

    return fig, axes


# Figure settings:
rows = 5
cols = 5
subfig_width = 4
subfig_height = 4
h_space = 0.1
v_space = 0.1
legend = True

# input
measuredDFRaw = power_rec_example_DF / overallNoiseFitResultDF.S.values
measuredDFRaw.index = np.asarray(measuredDFRaw.index.values) + 0.5
galaxySimulationDF = power_sim_avr_df
systemNoiseFitResultDF = overallNoiseFitResultDF
# lst=[5.5, 18.5]
lst = np.arange(24) + 0.5

# make
fig, axes = create_shared_subfigures(
    rows, cols, subfig_width, subfig_height, hspace=h_space, wspace=v_space
)

axes[0, 0].set_xlim(28, 82)
axes[0, 0].set_ylim(0, 40)

# Add x-axis labels only to the bottom row subfigures
for c in range(cols):
    axes[rows - 1, c].set_xlabel("frequency [MHz]")

# Add y-axis labels only to the leftmost column subfigures
for r in range(rows):
    axes[r, 0].set_ylabel("power [pW]")

i = 0
for r in range(rows):
    for c in range(cols):
        axes[r, c].plot([1, 2, 3, 4, 5], [r + c * 2] * 5)

        # try:
        # x-axis measured RAW
        frequencyRaw = measuredDFRaw.columns.values.astype(float)
        measuredNoiseRaw = measuredDFRaw.loc[lst[i], :].values
        # plot measured data
        axes[r, c].plot(
            frequencyRaw,
            measuredNoiseRaw,
            color="k",
            linewidth=2,
            label="cal. measured signal",
        )
        # except Exception as e:
        #     if i == 0:
        #         print('[INFO] No "measuredNoiseRaw".')
        #     pass

        try:
            # x-axis measured, baseline
            frequencyBaseline = measuredDFBaseline.columns.values.astype(float)
            measuredNoiseBaseline = measuredDFBaseline.loc[lst[i], :].values
            # plot measured data
            axes[r, c].plot(
                frequencyBaseline,
                measuredNoiseBaseline,
                color="g",
                linewidth=2,
                label="tot.signa(measured)\n-Baseline",
            )
        except Exception as e:
            # print(f"An error occurred: {e}")
            # if i == 0:
            #     print('[INFO] No "measuredNoiseBaseline".')
            pass

        try:
            # x-axis measured, baseline
            frequencyGalaxySimulation = galaxySimulationDF.columns.values.astype(float)
            galaxySignal = galaxySimulationDF.loc[lst[i], :].values
            # plot measured data
            axes[r, c].bar(
                frequencyGalaxySimulation,
                galaxySignal,
                alpha=1,
                color="orange",
                label="galactic signal",
            )
        except Exception as e:
            print(f"An error occurred: {e}")
            if i == 0:
                print('[INFO] No "galaxySimulationDF".')
            pass

        axes[r, c].bar(
            frequencyGalaxySimulation,
            overallNoiseFitResultDF.Nf.values,
            bottom=galaxySignal,
            color="b",
            label="thermal + external noise",
            alpha=0.5,
        )

        textstr = "LST: " + str(lst[i]) + " hour"
        props = dict(boxstyle="round", facecolor="wheat", alpha=0.5)

        axes[r, c].text(
            0.25,
            0.97,
            textstr,
            transform=axes[r, c].transAxes,
            fontsize=14,
            verticalalignment="top",
            bbox=props,
        )

        i += 1
        if i == len(lst):
            break

if legend:
    axes[-1, -2].legend(fontsize=12)

    label_params = axes[-1, -2].get_legend_handles_labels()

    axes[-1, -1].axis(False)
    axes[-1, -1].legend(
        *label_params,
        loc="upper left",
        bbox_to_anchor=(-0.1, 1.02),
        prop={"size": 14},
        fontsize=16,
    )

    axes[-1, -2].get_legend().remove()
    # plt.tight_layout()
    # plt.show()
    axes[-1, -1].xaxis.set_major_locator(MultipleLocator(10))

## Example of calibration methods

In [ ]:
def show_calibration_methods(df1, df2, col_start=0, pick_every=4, method='band2band', title=True):

    # fit each band separately but we a common slope
    x_data_set = df1.iloc[:,col_start:].iloc[:,pick_every::].T.values
    y_data_set = df2.iloc[:,col_start:].iloc[:,pick_every::].T.values

    # Initial guesses for the common slope and individual intercepts
    initial_params = [1.0] + [0.0] * len(x_data_set)  # [common slope, intercept1, intercept2, ...]

    def linear_model(params, x):
        slope = params[0]
        intercepts = params[1:]
        return [slope * x[i] + intercepts[i] for i in range(len(x))]

    # Define the objective function to minimize (sum of squared residuals)
    def objective(params):
        y_pred = linear_model(params, x_data_set)
        residuals = np.concatenate([(y - y_pred[i]) for i, y in enumerate(y_data_set)])
        return np.sum(residuals**2)

    # Perform the minimization
    result = minimize(objective, initial_params, method='Powell')

    # Extract the optimized common slope and individual intercepts
    common_slope = result.x[0]
    intercepts = result.x[1:]

    # print("Common Slope:", common_slope)
    # print("Intercepts:", intercepts)

    # Discretize 'jet' colormap
    cmap = plt.get_cmap("jet", df1.shape[1])

    fig, ax = plt.subplots(figsize=(6,8))

    # Set the minimum and maximum values for the colorbar
    vmin = 30  # Replace with your desired minimum value
    vmax = 80  # Replace with your desired maximum value
    norm = plt.Normalize(vmin, vmax)

    # Create a scatter plot for each pair of columns
    for i, f in enumerate(df1.columns[col_start:][pick_every::]):
        im = ax.scatter(
            df1.loc[:, f].values,
            df2.loc[:, f].values,
            c=np.ones(df1.index.size) * float(f),
            cmap=cmap,
            norm=norm,
            s=10
        )
        x  =  df1.loc[:, f].values
        y  =  df2.loc[:, f].values
        if method == 'band2band':
            # each band fitted totally independently
            q, k = robust_regression(x, y)
            ax.plot(x, k * x + q, color=cmap(norm(f)))
        elif method == 'band2band_common_slope':
            # each band separately but with a common slope
            y_fit = common_slope * x + intercepts[i]
            ax.plot(x, y_fit, color=cmap(norm(f)))
        else:
            pass
        
    if method == 'all-data-fit':
        # one fit across all bands
        x_mean = df1.values.flatten()
        y_mean = df2.values.flatten()
        q_mean, k_mean = robust_regression(x_mean, y_mean)
        ax.plot(x_mean[x_mean>2.2], k_mean * x_mean[x_mean>2.2] + q_mean, color='black', linestyle='-', lw=5)

    cbar = plt.colorbar(im, ax=ax, orientation="vertical", label="frequency [MHz]")

    ax.set_ylim(10, 27)
    ax.set_xlim(2, 13)

    ax.set_xlabel("predicted power [pW]")
    ax.set_ylabel("measured power [pW]")

    if title:
        ax.set_title("Calibration Method:\n{}".format(method))


In [ ]:
# some global plot settings
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["font.size"] = 20
plt.rcParams["legend.fontsize"] = 14

plt.rcParams["xtick.major.width"] = 3
plt.rcParams["ytick.major.width"] = 3

plt.rcParams["xtick.major.size"] = 6
plt.rcParams["ytick.major.size"] = 6

plt.rcParams["xtick.labelsize"] = 16
plt.rcParams["ytick.labelsize"] = 16

df2 = power_rec_example_DF
df1 = concatenated_sim_df.loc["Salla_EW_LFmap"]

col_start = 0
pick_every = 4

show_calibration_methods(df1, df2, col_start=0, pick_every=4, method='all-data-fit')
show_calibration_methods(df1, df2, col_start=0, pick_every=4, method='band2band_common_slope')
show_calibration_methods(df1, df2, col_start=0, pick_every=4, method='band2band')